In [1]:
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import torch
import numpy as np

import matplotlib.colors as mcolors
from matplotlib import cm
import seaborn as sns
import cv2

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from matplotlib import cm
import cv2
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [3]:
def count_numeric_subfolders(folder_path):
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"エラー: 指定されたパスが見つかりません: {folder_path}")
    if not os.path.isdir(folder_path):
        raise NotADirectoryError(f"エラー: 指定されたパスはフォルダではありません: {folder_path}")
    numeric_folder_count = 0
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path) and item.isdigit():
            numeric_folder_count += 1
    return numeric_folder_count

def filter_co_occur(data, sample_name, data_len, max_co_occur, out_num):
    filted_data = []
    filted_sample_name = []
    filted_data_len = []
    for i in range(len(data)):
        compare = 0
        for j in range(len(data[i])):
            mutation = data[i][j].split(',')
            if compare < len(mutation):
                compare = len(mutation)
        if compare <= max_co_occur:
            filted_data.append(data[i])
            filted_sample_name.append(sample_name[i])
            filted_data_len.append(data_len[i])
        if len(filted_data) >= out_num:
            break
    return filted_data, filted_sample_name, filted_data_len

In [4]:
def import_mutation_paths(base_dir, strain):
    # ホームディレクトリを展開
    base_dir = os.path.expanduser(base_dir)
    strain_dir = os.path.join(base_dir, strain)

    # strain直下のファイルパスを確認
    file_paths = []
    file_path = os.path.join(strain_dir, f"mutation_paths_{strain}.tsv")
    if os.path.exists(file_path):
        file_paths.append(file_path)
    
    # strain/numサブディレクトリを探索
    else:
        if os.path.exists(strain_dir) and os.path.isdir(strain_dir):
            num_dirs = [d for d in os.listdir(strain_dir) if d.isdigit()]
            num_dirs.sort(key=int)  # 数字順にソート

            for num in num_dirs:
                file_path = os.path.join(strain_dir, num, f"mutation_paths_{strain}.tsv")
                if os.path.exists(file_path):
                    file_paths.append(file_path)

    if not file_paths:
        raise FileNotFoundError(f"mutation_paths_{strain}.tsvが{strain_dir}内に見つかりませんでした。")

    return file_paths


In [5]:
if __name__ == "__main__":
    # --- データ読み込み・前処理 ---
    #strains = ['B.1.1.7','P.1','BA.2','BA.1.1','BA.1','B.1.617.2','B.1.351','B.1.1.529']
    strains = ['P.1','B.1.1.529']
    out_num = 1000000
    dir = '~/usher_output/'
    max_co_occur = 5

    # 全件データの読み込み
    names = []
    lengths = []
    paths = []
    for strain in strains:
        file_paths = import_mutation_paths(dir,strain)
        for file_path in file_paths:
            print(f"[INFO]import: {file_path}")
            f = open(file_path, 'r',encoding="utf-8_sig")
            datalist = f.readlines()
            f.close()

            data_num = 10 #or len(datalist)

            for i in range(1,data_num):
                data = datalist[i].split('\t')
                names.append(data[0])
                lengths.append(int(data[1]))
                paths.append(data[2].rstrip().split('>'))
        
    print(f"[INFO] 全件読み込み完了: {len(paths)} サンプル")
    filtered_paths, filtered_name, filtered_length = filter_co_occur(paths, names, lengths, max_co_occur, out_num)
    print(f"[INFO] 共起数フィルタリング完了: {len(filtered_paths)} サンプル")

[INFO]import: /mnt/ssd1/home3/aiba/usher_output/P.1/mutation_paths_P.1.tsv
[INFO]import: /mnt/ssd1/home3/aiba/usher_output/B.1.1.529/mutation_paths_B.1.1.529.tsv
[INFO] 全件読み込み完了: 18 サンプル
[INFO] 共起数フィルタリング完了: 0 サンプル


In [6]:
#変異パターンに対応する表中のセルのインクリメント（position step指定可能）
def count_mutation(df,mutation,step):

    bef = mutation[0]
    aft = mutation[-1]
    pos = int(int(mutation[1:-1]) /step + 0.5)
    pattern = f"{bef}->{aft}"

    df.loc[pos-1, pattern] += 1

# 変異パターンの表を作成
def mutation_table(paths,step=1):
    columns = ["position","A->T","A->G","A->C","T->A","T->G","T->C","G->A","G->T","G->C","C->A","C->T","C->G"]
    df = pd.DataFrame(np.zeros((int(30000/step),len(columns)),dtype=int), columns=columns)
    df["position"] = range(1, 30001, step)

    for path in paths:
        for mutations in path:
            for mutation in mutations.split(','):
                count_mutation(df, mutation,step)
    df_all = pd.DataFrame(np.zeros((int(30000/step), 2), dtype=int), columns=["position","all"])
    df_all["position"] = range(1, 30001, step)
    df_all["all"] = df.iloc[:, 1:].sum(axis=1)
    return df,df_all

# タイムステップごとの変異パターンの表を作成（position step指定可能）
def mutation_table_by_timestep(paths,max, step=1):
    df_timestep = []
    for i in range(max):
        columns = ["position","A->T","A->G","A->C","T->A","T->G","T->C","G->A","G->T","G->C","C->A","C->T","C->G"]
        df = pd.DataFrame(np.zeros((int(30000/step),len(columns)),dtype=int), columns=columns)
        df["position"] = range(1, 30001, step)

        for path in paths:
            if i >= len(path):
                continue
            for mutation in path[i].split(','):
                count_mutation(df, mutation, step)
        df_timestep.append(df)

    return df_timestep

In [7]:
# 利用可能なCPUコア数を取得
def get_cpu_count():
    return os.cpu_count()

# 実行例
print(f"利用可能なCPUコア数: {get_cpu_count()}")

利用可能なCPUコア数: 16


In [8]:
# 並列処理版の変異パターンの表を作成
def mutation_table_parallel(paths, step=1):
    columns = ["position","A->T","A->G","A->C","T->A","T->G","T->C","G->A","G->T","G->C","C->A","C->T","C->G"]
    df = pd.DataFrame(np.zeros((int(30000/step),len(columns)),dtype=int), columns=columns)
    df["position"] = range(1, 30001, step)

    def process_path(path):
        local_df = pd.DataFrame(np.zeros((int(30000/step),len(columns)),dtype=int), columns=columns)
        local_df["position"] = range(1, 30001, step)
        for mutations in path:
            for mutation in mutations.split(','):
                count_mutation(local_df, mutation, step)
        return local_df

    with ThreadPoolExecutor(max_workers=8) as executor:
        results = []
        for result in tqdm(executor.map(process_path, paths), total=len(paths), desc="Processing paths"):
            results.append(result)

    for result in results:
        df.iloc[:, 1:] += result.iloc[:, 1:]

    df_all = pd.DataFrame(np.zeros((int(30000/step), 2), dtype=int), columns=["position","all"])
    df_all["position"] = range(1, 30001, step)
    df_all["all"] = df.iloc[:, 1:].sum(axis=1)
    return df, df_all

# 並列処理版のタイムステップごとの変異パターンの表を作成
def mutation_table_by_timestep_parallel(paths, max, step=1):
    columns = ["position","A->T","A->G","A->C","T->A","T->G","T->C","G->A","G->T","G->C","C->A","C->T","C->G"]

    def process_timestep(i):
        df = pd.DataFrame(np.zeros((int(30000/step),len(columns)),dtype=int), columns=columns)
        df["position"] = range(1, 30001, step)
        for path in paths:
            if i >= len(path):
                continue
            for mutation in path[i].split(','):
                count_mutation(df, mutation, step)
        return df

    with ThreadPoolExecutor(max_workers=8) as executor:
        df_timestep = []
        for df in tqdm(executor.map(process_timestep, range(max)), total=max, desc="Processing timesteps"):
            df_timestep.append(df)

    return df_timestep

In [9]:
# DataFrameをCSVファイルに保存
def save_df(df, output_dir = 'tables', output_file_name = 'mutation_table.csv'):

    os.makedirs(output_dir, exist_ok=True)

    file_path = os.path.join(output_dir, output_file_name)
    df.to_csv(file_path, index=False)
    #print(f"Saved: {file_path}")

# タイムステップごとのDataFrameをCSVファイルに保存
def save_timestep_df(timestep_dataframes, output_dir = 'timestep_tables'):

    os.makedirs(output_dir, exist_ok=True)

    for i, df in enumerate(timestep_dataframes):
        file_path = os.path.join(output_dir, f"timestep_{i + 1}.csv")
        df.to_csv(file_path, index=False)
        #print(f"Saved: {file_path}")

# タイムステップごとのヒートマップを作成・保存
def save_heatmaps(df_timestep, step, output_dir = 'timestep_heatmaps'):
    viridis = cm.get_cmap('viridis', 256)
    newcolors = viridis(np.linspace(0, 1, 256))
    newcolors[0, :] = np.array([1, 1, 1, 1])  # 0番目を白に
    white_viridis = mcolors.ListedColormap(newcolors)

    os.makedirs(output_dir, exist_ok=True)

    columns = ["position","A->T","A->G","A->C","T->A","T->G","T->C","G->A","G->T","G->C","C->A","C->T","C->G"]
    x_labels = columns[1:]  # positionを除外

    for i, df in enumerate(df_timestep):
        data = df.iloc[:, 1:]
        norm_data = (data - data.min().min()) / (data.max().max() - data.min().min())
        plt.figure(figsize=(10, 6))
        plt.imshow(norm_data, aspect='auto', cmap=white_viridis, interpolation='nearest', vmin=0, vmax=1)
        plt.colorbar(label='Normalized Value')
        plt.title(f'Timestep {i+1}')
        plt.xlabel('Mutation Type')
        plt.ylabel(f'position')
        plt.xticks(ticks=np.arange(len(x_labels)), labels=x_labels, rotation=45, ha='right')
        plt.yticks(ticks=np.arange(0, len(df) + 1, step // 2), labels=(np.arange(0, len(df) + 1, step // 2) * step).astype(int))
        plt.tight_layout()
        plt.savefig(f'{output_dir}/mutation_heatmap_timestep_{i+1}.png')
        plt.close()
    print('ヒートマップを保存しました。')

# タイムステップごとのヒートマップ画像から動画を作成
def create_video_from_heatmaps(image_folder, output_video_path="timestep_heatmaps/mutation_heatmaps_video.mp4", fps=2):
    # Get list of image files sorted by timestep
    images = [img for img in os.listdir(image_folder) if img.endswith(".png") and "heatmap_timestep" in img]
    images.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Sort by timestep

    # Read the first image to get dimensions
    if not images:
        raise ValueError("No valid heatmap images found in the specified folder.")

    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Add images to the video
    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        video.write(frame)

    video.release()

In [10]:
output_dir = "table_heatmap0002"
timestep_max = max(lengths)

use_sample_num = len(names)

In [11]:
pos_step = 1    #可視化するなら100以上、しないなら1
df, df_all = mutation_table_parallel(paths[:use_sample_num],pos_step)
save_df(df, output_dir, output_file_name = 'mutation_table_pattern_pos-step'+str(pos_step)+'.csv')
save_df(df_all, output_dir, output_file_name = 'mutation_table_all_pos-step'+str(pos_step)+'.csv')

pos_step = 100
df_timestep = mutation_table_by_timestep_parallel(paths[:use_sample_num],timestep_max,pos_step)
tables_dir = os.path.join(output_dir, "timestep_tables_pos-step" + str(pos_step))
heatmaps_dir = os.path.join(output_dir, "timestep_heatmaps_pos-step" + str(pos_step))
save_timestep_df(df_timestep, tables_dir)
save_heatmaps(df_timestep, pos_step, heatmaps_dir)
create_video_from_heatmaps(heatmaps_dir, heatmaps_dir+"/mutation_heatmaps_video.mp4", fps=1)

Processing timesteps: 100%|██████████| 29/29 [00:00<00:00, 267329.27it/s]
/tmp/ipykernel_37035/4198424693.py:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  viridis = cm.get_cmap('viridis', 256)


ヒートマップを保存しました。


In [12]:
#mutation-pathsを転置して重複を除去
def transposition_and_set(paths):
    timestep_mutation = {}

    for path in paths:
        for timestep, sample in enumerate(path):
            if timestep not in timestep_mutation:
                timestep_mutation[timestep] = set()
            timestep_mutation[timestep].add(sample)

    return timestep_mutation

#タイムステップごとのリストを1次リストに変換
def to1Dlist(timestep_data):

    mutation_list = []
    
    for timestep, mutations in timestep_data.items():
        #print(mutations)
        for mutation in mutations:
            #rint(mutation)
            mutation_list.append(mutation)
    
    return mutation_list

# 重複を除去した1次リストから変異パターンの表を作成（position step指定可能）
def mutation_table_form_1Dlist(path,step=1):
    columns = ["position","A->T","A->G","A->C","T->A","T->G","T->C","G->A","G->T","G->C","C->A","C->T","C->G"]
    df = pd.DataFrame(np.zeros((int(30000/step),len(columns)),dtype=int), columns=columns)
    df["position"] = range(1, 30001, step)

    for mutations in path:
        for mutation in mutations.split(','):
            count_mutation(df, mutation,step)
    df_all = pd.DataFrame(np.zeros((int(30000/step), 2), dtype=int), columns=["position","all"])
    df_all["position"] = range(1, 30001, step)
    df_all["all"] = df.iloc[:, 1:].sum(axis=1)
    return df,df_all

In [13]:
pos_step = 1
timestep_data = transposition_and_set(paths[:use_sample_num])
mutation_list = to1Dlist(timestep_data)
df_set,df_set_all = mutation_table([mutation_list],pos_step)
save_df(df_set, output_dir, output_file_name = 'mutation_table_set_pattern_pos-step'+str(pos_step)+'.csv')
save_df(df_set_all, output_dir, output_file_name = 'mutation_table_set_all_pos-step'+str(pos_step)+'.csv')

pos_step = 100
timestep_data = transposition_and_set(paths[:use_sample_num])
mutation_list = to1Dlist(timestep_data)
df_set,df_set_all = mutation_table([mutation_list],pos_step)
heatmaps_set_dir = os.path.join(output_dir, "timestep_heatmaps_set_pos-step" + str(pos_step))
save_df(df_set, heatmaps_set_dir, output_file_name = 'mutation_table_set_pattern_pos-step'+str(pos_step)+'.csv')
save_heatmaps([df_set],pos_step,heatmaps_set_dir)

ヒートマップを保存しました。


/tmp/ipykernel_37035/4198424693.py:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  viridis = cm.get_cmap('viridis', 256)


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

# 並列処理のテスト関数
def test_function(index):
    print(f"タスク {index} 開始")
    time.sleep(2)  # 2秒間スリープして並列性を確認
    print(f"タスク {index} 終了")
    return index

# 並列処理の確認プログラム
def check_parallel_execution():
    tasks = list(range(10))  # 10個のタスクを作成
    start_time = time.time()
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(test_function, tasks))
    end_time = time.time()
    print(f"結果: {results}")
    print(f"実行時間: {end_time - start_time} 秒")

# 実行例
check_parallel_execution()

タスク 0 開始
タスク 1 開始
タスク 2 開始
タスク 3 開始
タスク 4 開始
タスク 5 開始
タスク 6 開始
タスク 7 開始
タスク 8 開始
タスク 9 開始
タスク 0 終了
タスク 1 終了
タスク 2 終了
タスク 3 終了
タスク 4 終了
タスク 5 終了
タスク 6 終了
タスク 9 終了
タスク 7 終了
タスク 8 終了
結果: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
実行時間: 2.008362054824829 秒
タスク 0 終了
タスク 1 終了
タスク 2 終了
タスク 3 終了
タスク 4 終了
タスク 5 終了
タスク 6 終了
タスク 9 終了
タスク 7 終了
タスク 8 終了
結果: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
実行時間: 2.008362054824829 秒


In [ ]:
import time

# 並列処理を使用しないテスト関数
def test_function_sequential(index):
    print(f"タスク {index} 開始")
    time.sleep(2)  # 2秒間スリープ
    print(f"タスク {index} 終了")
    return index

# 並列処理を使用しない確認プログラム
def check_sequential_execution():
    tasks = list(range(10))  # 10個のタスクを作成
    start_time = time.time()
    results = []
    for task in tasks:
        results.append(test_function_sequential(task))
    end_time = time.time()
    print(f"結果: {results}")
    print(f"実行時間: {end_time - start_time} 秒")

# 実行例
check_sequential_execution()

タスク 0 開始
タスク 0 終了
タスク 1 開始
タスク 0 終了
タスク 1 開始
タスク 1 終了
タスク 2 開始
タスク 1 終了
タスク 2 開始
タスク 2 終了
タスク 3 開始
タスク 2 終了
タスク 3 開始
タスク 3 終了
タスク 4 開始
タスク 3 終了
タスク 4 開始
タスク 4 終了
タスク 5 開始
タスク 4 終了
タスク 5 開始
タスク 5 終了
タスク 6 開始
タスク 5 終了
タスク 6 開始
タスク 6 終了
タスク 7 開始
タスク 6 終了
タスク 7 開始
タスク 7 終了
タスク 8 開始
タスク 7 終了
タスク 8 開始
タスク 8 終了
タスク 9 開始
タスク 8 終了
タスク 9 開始
タスク 9 終了
結果: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
実行時間: 20.022986888885498 秒
タスク 9 終了
結果: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
実行時間: 20.022986888885498 秒
